In [52]:
import os
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
import sklearn.decomposition
import mdtraj 
import random
import plotly.graph_objs as go
import pickle


In [53]:
with open('tr.pickle', 'rb') as f:
    tr = pickle.load(f)


In [57]:
def dict_coord(tr):
    coord={}
    for key in tr.keys():
        ca_ids = tr[key].topology.select("protein and name CA")
        ca_xyz = tr[key].xyz[:,ca_ids]
        coord[key]=ca_xyz
    return coord
ca_xyz_dict=dict_coord(tr)
def compute_cos_consecutive(ca_xyz_dict):
    computed_data = {}
    for key, ca_xyz in ca_xyz_dict.items():
        starting_matrix = np.zeros((ca_xyz.shape[0], ca_xyz.shape[1]-1, ca_xyz.shape[1]-1))
        for conformation in range(ca_xyz.shape[0]):
            for i in range(ca_xyz.shape[1]-1):
                vector_i_i_plus1 = ca_xyz[conformation][i+1] - ca_xyz[conformation][i]
                vector_i_i_plus1 /= np.linalg.norm(vector_i_i_plus1)
                for j in range(ca_xyz.shape[1]-1):
                    vector_j_j_plus1 = ca_xyz[conformation][j+1] - ca_xyz[conformation][j]
                    vector_j_j_plus1 /= np.linalg.norm(vector_j_j_plus1)
                    cos_i_j = np.dot(vector_i_i_plus1, vector_j_j_plus1)
                    starting_matrix[conformation][i][j] = cos_i_j
        computed_data[key] = starting_matrix
    return computed_data

cos_dict=compute_cos_consecutive(ca_xyz_dict)
def site_pair_correlation_parameter(cos_dict):
    dict_K_ij={}
    dict_o_i={}
    for key  in  cos_dict.keys():
        mean_cos_tetha=np.mean(cos_dict[key],axis=0)
        square_diff=(cos_dict[key]-mean_cos_tetha)**2
        variance_cos_tetha=np.mean(square_diff,axis=0)
        K_ij = [[1 - np.sqrt(2 * variance_cos_tetha[i][j]) for j in range(variance_cos_tetha.shape[1])] for i in range(variance_cos_tetha.shape[0])]
        dict_K_ij[key]=np.array(K_ij)
    for key in dict_K_ij.keys():
        o_i=np.mean(dict_K_ij[key],axis=1)
        dict_o_i[key]=np.array(o_i)
    return dict_o_i
dict_o_i=site_pair_correlation_parameter(cos_dict)
def plot_o_i(dict_o_i):
    fig = go.Figure()
    for key, o_i in dict_o_i.items():
        position = list(range(o_i.shape[0])) 
        trace = go.Scatter(x=position, y=o_i, mode='markers', name=key)
        fig.add_trace(trace)
    
    layout = go.Layout(
        title='Site-specific order parameter',
        xaxis=dict(title='Position'),
        yaxis=dict(title='Value', range=[0.14, 0.26]),
        width=1000,
        height=600
    )
    
    fig.update_layout(layout)
    
    for key, _ in dict_o_i.items():
        fig.add_shape(type="line", x0=min(position), y0=0.1906, x1=max(position), y1=0.1906, line=dict(color="black", width=7))
    
    fig.show()
plot_o_i(dict_o_i)


In [58]:
def dict_coord(tr):
    coord={}
    for key in tr.keys():
        ca_ids = tr[key].topology.select("protein and name CA")
        ca_xyz = tr[key].xyz[:,ca_ids]
        coord[key]=ca_xyz
    return coord


def site_specific_order_parameter(ca_xyz_dict):
    computed_data = {}
    for key, ca_xyz in ca_xyz_dict.items():
        starting_matrix = np.zeros((ca_xyz.shape[0], ca_xyz.shape[1]-1, ca_xyz.shape[1]-1))
        for conformation in range(ca_xyz.shape[0]):
            for i in range(ca_xyz.shape[1]-1):
                vector_i_i_plus1 = ca_xyz[conformation][i+1] - ca_xyz[conformation][i]
                vector_i_i_plus1 /= np.linalg.norm(vector_i_i_plus1)
                for j in range(ca_xyz.shape[1]-1):
                    vector_j_j_plus1 = ca_xyz[conformation][j+1] - ca_xyz[conformation][j]
                    vector_j_j_plus1 /= np.linalg.norm(vector_j_j_plus1)
                    cos_i_j = np.dot(vector_i_i_plus1, vector_j_j_plus1)
                    starting_matrix[conformation][i][j] = cos_i_j
        mean_cos_tetha = np.mean(starting_matrix, axis=0)
        square_diff = (starting_matrix - mean_cos_tetha) ** 2
        variance_cos_tetha = np.mean(square_diff, axis=0)
        K_ij = np.array([[1 - np.sqrt(2 * variance_cos_tetha[i][j]) for j in range(variance_cos_tetha.shape[1])] for i in range(variance_cos_tetha.shape[0])])
        o_i = np.mean(K_ij, axis=1)
        computed_data[key] = o_i
    return computed_data
dict_o_i=site_specific_order_parameter(ca_xyz_dict)
def plot_o_i(dict_o_i):
    fig = go.Figure()
    for key, o_i in dict_o_i.items():
        position = list(range(o_i.shape[0])) 
        trace = go.Scatter(x=position, y=o_i, mode='markers', name=key)
        fig.add_trace(trace)
    
    layout = go.Layout(
        title='Site-specific order parameter',
        xaxis=dict(title='Position'),
        yaxis=dict(title='Value', range=[0.14, 0.26]),
        width=1000,
        height=600
    )
    
    fig.update_layout(layout)
    
    for key, _ in dict_o_i.items():
        fig.add_shape(type="line", x0=min(position), y0=0.1906, x1=max(position), y1=0.1906, line=dict(color="black", width=7))
    
    fig.show()
plot_o_i(dict_o_i)

